In [1]:
import pypowsybl.network as pn
network = pn.load('/home/dupuyflo/Data/LFE_2025/fr_0700.xiidm')

In [2]:
voltage_levels = network.get_voltage_levels(attributes=['nominal_v'])
vl_400kV_ids = voltage_levels[voltage_levels['nominal_v'] > 300].index
lines = network.get_lines(attributes=['voltage_level1_id', 'voltage_level2_id'])
lines_400kV = lines[
    lines['voltage_level1_id'].isin(vl_400kV_ids) &
    lines['voltage_level2_id'].isin(vl_400kV_ids)
]
lines_400kV

,voltage_level1_id,voltage_level2_id
id,,
AGASSL71JONQU,AGASSP7,JONQUP7
AGASSL71TAVEL,AGASSP7,TAVELP7
ALBERL71BATHI,ALBERP7,BATHIP7
ALBERL71COCHE,ALBERP7,COCHEP7
ALBERL71G.ILE,ALBERP7,G.ILEP7
...,...,...
TERR6L72TOURB,TERR6P7,TOURBP7
VERGEL71VLERB,VERGEP7,VLERBP7
VERGEL72VLERB,VERGEP7,VLERBP7


In [3]:
import pypowsybl.security as ps
security_analysis = ps.create_analysis()
security_analysis.add_single_element_contingencies(lines_400kV.index.to_list());
result = security_analysis.run_ac(network)

In [4]:
lv_post_contingency = result.limit_violations.copy().reset_index()
lv_post_contingency = lv_post_contingency[lv_post_contingency['contingency_id'] != '']
lv_post_contingency

,contingency_id,subject_id,subject_name,limit_type,limit_name,limit,acceptable_duration,limit_reduction,value,side
31,CHAFFL72P.COR,BEZAUP1,,LOW_VOLTAGE,,19.000000,2147483647,1.0,18.952749,
32,CHAFFL72P.COR,1FRAS77,,HIGH_VOLTAGE,,420.000031,2147483647,1.0,423.705521,
33,CHAFFL72P.COR,FRASNP1,,HIGH_VOLTAGE,,22.000000,2147483647,1.0,22.300291,
34,CHAFFL72P.COR,VALDIP1,,LOW_VOLTAGE,,19.000000,2147483647,1.0,18.839923,
35,CHAFFL72P.COR,VLEVAP1,,LOW_VOLTAGE,,19.000000,2147483647,1.0,18.927637,
...,...,...,...,...,...,...,...,...,...,...
5832,GRAV5L76WARAN,VALDIP1,,LOW_VOLTAGE,,19.000000,2147483647,1.0,18.839967,
5833,GRAV5L76WARAN,SSELOP1,,LOW_VOLTAGE,,19.000000,2147483647,1.0,18.900304,
5834,GRAV5L76WARAN,CSTTTP1,,HIGH_VOLTAGE,,21.000000,2147483647,1.0,21.492972,
5835,GRAV5L76WARAN,L.ATHP1,,HIGH_VOLTAGE,,21.000000,2147483647,1.0,21.435828,


In [5]:
violations_by_equipment = (
    lv_post_contingency
    .groupby(['subject_id', 'limit_type'])['contingency_id']
    .apply(list)
    .reset_index()
)
violations_line = violations_by_equipment[violations_by_equipment['limit_type']=='CURRENT']
violations_vl = violations_by_equipment[violations_by_equipment['limit_type']!='CURRENT']
violations_line

,subject_id,limit_type,contingency_id
2,ALBERL61LONG6,CURRENT,"[COCHEL71P.AND, COCHEL71P.AND, ALBERL71COCHE, ..."
4,ARBOIL31CPNOL,CURRENT,"[FRASNL71MAMBE, FRASNL71MAMBE, FRASNL71GEN.P, ..."
5,ARBOIL31CPVAN,CURRENT,"[FRASNL71GEN.P, FRASNL71GEN.P]"
27,CPNOLL61SAONE,CURRENT,"[FRASNL71MAMBE, FRASNL71MAMBE]"
28,CPVANL31MESNA,CURRENT,"[FRASNL71GEN.P, FRASNL71GEN.P]"
29,CPVANL31ZBRE6,CURRENT,"[FRASNL71GEN.P, FRASNL71GEN.P]"
61,LONG6L21ZGLAN,CURRENT,"[COCHEL71P.AND, COCHEL71P.AND, ALBERL71COCHE, ..."
70,MESNAL31ZMESN,CURRENT,"[FRASNL71GEN.P, FRASNL71GEN.P]"
73,P.ANDL61SAUS2,CURRENT,"[COCHEL71P.AND, COCHEL71P.AND, ALBERL71COCHE, ..."
75,P.ANDY761,CURRENT,"[COCHEL71P.AND, ALBERL71COCHE]"


In [7]:
import pandas as pd

default_pf = network.get_default_nad_profile()

vl_descriptions_df = default_pf.vl_descriptions[default_pf.vl_descriptions['type'] != 'FOOTER']

labels_df = default_pf.branch_labels
labels_df['middle'] = ''
contingency_series = violations_line['contingency_id'].apply(lambda x: ','.join(x))
contingency_series.index = violations_line['subject_id']
labels_df.loc[violations_line['subject_id'], 'middle'] = contingency_series

edges_styles_df = pd.DataFrame({
    'id': violations_line['subject_id'],
    'edge1': 'pink',
    'edge2': 'pink',
    'width1': '150px',
    'width2': '150px'
})
edges_styles_df.set_index('id', inplace=True)

diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=default_pf.bus_descriptions,
                              bus_node_styles=default_pf.bus_node_styles, edge_styles=edges_styles_df)


In [8]:
metadata = pd.read_json('/home/dupuyflo/Documents/LFE_2025/france_metadata.json')
fixed_positions = pd.DataFrame({
    "id": metadata["equipmentId"],
    "x": metadata["x"],
    "y": metadata["y"]
})
fixed_positions.set_index('id', drop=True, inplace=True)

In [9]:
from pypowsybl_jupyter import network_explorer

network_explorer(network, depth=3, vl_id='GEN.PP6', fixed_nad_positions=fixed_positions, nad_profile=diagram_profile)

In [10]:
violations_vl['contingencies'] = violations_vl['contingency_id'].apply(lambda x: ','.join(x))
violations_vl.index = violations_vl['subject_id']

color_mapping = {
    'HIGH_VOLTAGE': 'orange',
    'LOW_VOLTAGE': 'purple',
}

bus_vl_id = network.get_buses(attributes=['voltage_level_id'])
bus_vl_id['descr'] = bus_vl_id['voltage_level_id'].map(violations_vl['contingencies'])
bus_vl_id['edge-width'] = '100px'
bus_styles_df = bus_vl_id[bus_vl_id['descr'].notnull()][['edge-width']]
bus_styles_df['limit_type'] = bus_vl_id['voltage_level_id'].map(violations_vl['limit_type'])
bus_styles_df['edge'] = bus_styles_df['limit_type'].map(color_mapping)
bus_styles_df['fill'] = bus_styles_df['edge']
del bus_styles_df['limit_type']

In [11]:
diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=pd.DataFrame(),
                              bus_node_styles=bus_styles_df, edge_styles=edges_styles_df)

network_explorer(network, depth=3, vl_id='GEN.PP6', fixed_nad_positions=fixed_positions, nad_profile=diagram_profile)

In [11]:
result.limit_violations

subject_name    limit_type limit_name       limit  \
contingency_id subject_id                                                     
               BEZAUP1                   LOW_VOLTAGE              19.000000   
               1FRAS77                  HIGH_VOLTAGE             420.000031   
               FRASNP1                  HIGH_VOLTAGE              22.000000   
               HOURAP1                  HIGH_VOLTAGE              21.000000   
               MAZURP1                  HIGH_VOLTAGE              22.000000   
...                                ...           ...        ...         ...   
GRAV5L76WARAN  VALDIP1                   LOW_VOLTAGE              19.000000   
               SSELOP1                   LOW_VOLTAGE              19.000000   
               CSTTTP1                  HIGH_VOLTAGE              21.000000   
               L.ATHP1                  HIGH_VOLTAGE              21.000000   
               TRI.PP6                   LOW_VOLTAGE             227.000000   

                           acceptable_duration  limit_reduction       value  \
contingency_id subject_id                                                     
               BEZAUP1              2147483647              1.0   18.952958   
               1FRAS77              2147483647              1.0  423.669355   
               FRASNP1              2147483647              1.0   22.298387   
               HOURAP1              2147483647              1.0   21.593424   
               MAZURP1              2147483647              1.0   22.595634   
...                                        ...              ...         ...   
GRAV5L76WARAN  VALDIP1              2147483647              1.0   18.839967   
               SSELOP1              2147483647              1.0   18.900304   
               CSTTTP1              2147483647              1.0   21.492972   
               L.ATHP1              2147483647              1.0   21.435828   
               TRI.PP6              2147483647              1.0  224.341875   

                          side  
contingency_id subject_id       
               BEZAUP1          
               1FRAS77          
               FRASNP1          
               HOURAP1          
               MAZURP1          
...                        ...  
GRAV5L76WARAN  VALDIP1          
               SSELOP1          
               CSTTTP1          
               L.ATHP1          
               TRI.PP6          

[5837 rows x 8 columns]